In [2]:
# !pip install pythreejs
#!pip install numpy, scipy
# !pip install notebook==6.5.4
# !pip install jupyter_contrib_nbextensions
# !jupyter contrib nbextension install --user

In [106]:
from pythreejs import * 
from IPython.display import display

import ipywidgets as widgets
import numpy as np
import time
from scipy.spatial.transform import Rotation as R

In [119]:
def generate_cube(width=1.0, height=1.0, depth=1.0):
    """Generate vertices and faces for a box."""
    vertices = np.array([
        [-width/2, -height/2, -depth/2],  # 0 - Back Bottom Left
        [ width/2, -height/2, -depth/2],  # 1 - Back Bottom Right
        [ width/2,  height/2, -depth/2],  # 2 - Back Top Right
        [-width/2,  height/2, -depth/2],  # 3 - Back Top Left
        [-width/2, -height/2,  depth/2],  # 4 - Front Bottom Left
        [ width/2, -height/2,  depth/2],  # 5 - Front Bottom Right
        [ width/2,  height/2,  depth/2],  # 6 - Front Top Right
        [-width/2,  height/2,  depth/2],  # 7 - Front Top Left
        [0, 0, 0] # 8
    ], dtype=np.float32)

    # Define 12 triangles (6 faces x 2 triangles per face)
    faces = np.array([
        [4, 5, 6], [4, 6, 7],  # Front (+Z)
        [1, 0, 3], [1, 3, 2],  # Back (-Z)
        [5, 1, 2], [5, 2, 6],  # Right (+X)
        [0, 4, 7], [0, 7, 3],  # Left (-X)
        [3, 7, 6], [3, 6, 2],  # Top (+Y)
        [0, 1, 5], [0, 5, 4]   # Bottom (-Y)
    ], dtype=np.uint32)
    
    # Define normals per triangle face (each normal repeats for 2 triangles per face)
    normals = np.array([
        [-0.577, -0.577, -0.577],  # 0 - Averaged from Back, Bottom, Left
        [ 0.577, -0.577, -0.577],  # 1 - Averaged from Back, Bottom, Right
        [ 0.577,  0.577, -0.577],  # 2 - Averaged from Back, Top, Right
        [-0.577,  0.577, -0.577],  # 3 - Averaged from Back, Top, Left
        [-0.577, -0.577,  0.577],  # 4 - Averaged from Front, Bottom, Left
        [ 0.577, -0.577,  0.577],  # 5 - Averaged from Front, Bottom, Right
        [ 0.577,  0.577,  0.577],  # 6 - Averaged from Front, Top, Right
        [-0.577,  0.577,  0.577]   # 7 - Averaged from Front, Top, Left
    ], dtype=np.float32)
    
    edges = np.array([
        [0, 1], [1, 2], [2, 3], [3, 0], # Back face
        [4, 5], [5, 6], [6, 7], [7, 4], # Front face
        [0, 4], [1, 5], [2, 6], [3, 7], # Connect front and back faces
        # [8, 0], [8, 1], [8, 2], [8, 3],  # Connect center to back face
        # [8, 4], [8, 5], [8, 6], [8, 7],   # Connect center to front face
    ], dtype=np.uint32)
    
    tet_indices = np.array([
        [8, 0, 1, 2], [8, 0, 2, 3], # Back face
        [8, 4, 5, 6], [8, 4, 6, 7], # Front face
        [8, 0, 4, 5], [8, 0, 5, 1], # Connect front and back faces
        [8, 2, 6, 7], [8, 2, 7, 3], # Connect front
        
    ], dtype=np.uint32)
    
    return vertices, faces, normals, edges, tet_indices

def draw_cube(width=1.0, height=1.0, depth=1.0, color='green', position=[0, 0, 0], rotation=[0, 0, 0]):
    """Draw a cube with a given size and color."""
    vertices, faces, normals, edges, tet_indices = generate_cube(width, height, depth)
    geometry = BufferGeometry(
        attributes={
            'position': BufferAttribute(array=vertices.copy(), normalized=False),
            'normal': BufferAttribute(array=normals.copy(), normalized=True),
        },
        index=BufferAttribute(array=faces.copy().ravel(), normalized=False)
    )
    material = MeshStandardMaterial(
        color=color,  # RGB color of the material
        flatShading=True,
        wireframe=False,
    )
    cube = Mesh(geometry=geometry, material=material)
    cube.castShadow = True
        
    cube.init_position = cube.geometry.attributes['position'].array.copy()
    cube.init_rotation = R.from_euler('xyz', rotation, degrees=True)
    
    cube.geometry.attributes['position'].array = np.array(position, dtype=np.float32) +  cube.init_rotation.apply(cube.init_position.copy())
    
    cube.edges = edges
    cube.tet_indices = tet_indices
    return cube

def draw_plane(origin=[0, 0, 0], normal=[0, 1, 0], size=10, color='gray', opacity=0.5):
    """Draw a plane with a given normal and size."""
    # Define the plane geometry
    plane_geometry = PlaneGeometry(width=size, height=size)
    plane_material = MeshStandardMaterial(
        color=color, 
        transparent=True, 
        opacity=opacity, 
        side='DoubleSide')
    plane = Mesh(geometry=plane_geometry, material=plane_material)
    plane.receiveShadow = True
    
    # Rotate the plane to the normal vector
    default_normal = np.array([0, 0, 1])  # PlaneGeometry default normal
    axis = np.cross(default_normal, normal)
    angle = np.arccos(np.dot(default_normal, normal))  # Angle between vectors
    
    if np.linalg.norm(axis) > 1e-6:  # Avoid division by zero if vectors are parallel
        axis = axis / np.linalg.norm(axis)
        plane.quaternion = list(np.append(axis * np.sin(angle / 2), np.cos(angle / 2)))  # Quaternion rotation 
    
    plane.position = origin
    return plane

# World Setup

In [120]:
# Rendering setup
width, height = 800, 600 
aspect = width / height
scene = Scene(background='skyblue')  # Background color of the scene
camera = PerspectiveCamera(
    position=[5, 5, 10], 
    lookAt=[0, 0, 0],
    aspect=aspect
    )

ambient_light = AmbientLight(color='white', intensity=1.0)  # Soft overall light
directional_light = DirectionalLight(
    color='white',
    intensity=2.0,        # Brightness
    position=[10, 10, 0],   # Light direction
    
)
spot_light = SpotLight(
    color='white',
    intensity=5.0,        # Increase brightness
    position=[0, 5, 0],   # Place the light above the cube
    distance=10,          # Light range (covers cube)
    angle=0.5,            # Cone angle (medium focus)
    penumbra=0.3,         # Soft shadow edges
    decay=2,              # Realistic falloff
)
spot_light.castShadow = True  # Enable shadow casting
scene.add([ambient_light, directional_light])

controller = OrbitControls(controlling=camera, enableRotate=True)
renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=width, height=height)
renderer.shadowMap.enabled = True
renderer.shadowMap.type = 'PCFSoftShadowMap'


# Scene setup


cube1 = draw_cube(width=1, height=3, depth=1, color='green', position=[0, 4.5, 0])
cube2 = draw_cube(width=1, height=3, depth=1, color='blue', position=[0, 2.5, 0])
cube3 = draw_cube(width=1, height=3, depth=1, color='red', position=[1.0, 0.5, 0], rotation=[0, 0, 90])

scene.add(cube1)
scene.add(cube2)
scene.add(cube3)

plane = draw_plane(normal=[0, 1, 0], size=50, color='gray', opacity=1.0)
scene.add(plane)

button = widgets.Button(description="RESET")
output = widgets.Output()
def on_button_clicked(b):
    cube1.geometry.attributes['position'].array = cube1.init_position.copy()

button.on_click(on_button_clicked)


display(widgets.VBox([renderer, button, output]))

c:\Users\bluec\AppData\Local\Programs\Python\Python312\Lib\site-packages\pythreejs\traits.py:257: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


In [89]:

def solve_volume_constraint(x1, x2, x3, x4, w1, w2, w3, w4, rest_volume, compliance, h, prev_lambda=None):
    """
    Solves the volume conservation constraint for a tetrahedron.
    """
    # Compute current volume using the triple scalar product
    volume = np.dot(np.cross(x2 - x1, x3 - x1), (x4 - x1))

    C = volume - 6*rest_volume  # Volume constraint violation
    
    # If the volume is nearly zero, avoid instability
    if abs(volume) < 1e-6:
        return np.zeros_like(x1), np.zeros_like(x2), np.zeros_like(x3), np.zeros_like(x4), 0

    # Compute the gradients for all four vertices
    grad_x1 = np.cross(x4 - x2, x3 - x1) 
    grad_x2 = np.cross(x3 - x1, x4 - x2)
    grad_x3 = np.cross(x4 - x1, x2 - x1)
    grad_x4 = np.cross(x2 - x1, x3 - x1) 

    # Compute the sum of weighted gradients
    weighted_sum = w1 * np.dot(grad_x1, grad_x1) + w2 * np.dot(grad_x2, grad_x2) \
                 + w3 * np.dot(grad_x3, grad_x3) + w4 * np.dot(grad_x4, grad_x4)

    # Compliance term (XPBD-style)
    alpha = compliance / (h * h)
    
    # Solve for Lagrange multiplier update
    dlambda = - (C + alpha * prev_lambda) / (weighted_sum + alpha)
    new_lambda = prev_lambda + dlambda

    # Compute displacement updates for each vertex
    dx1 = -w1 * dlambda * grad_x1
    dx2 = -w2 * dlambda * grad_x2
    dx3 = -w3 * dlambda * grad_x3
    dx4 = -w4 * dlambda * grad_x4

    return dx1, dx2, dx3, dx4, new_lambda

def solve_distance_constraint(x1, x2, w1, w2, rest_length, compliance, h, prev_lambda=None):
    n = (x1 - x2)
    d = np.linalg.norm(n)
    # C(x1, x2) = |x1 - x2| - rest_length
    C = d - rest_length 
    
    if d < 1e-6:
        return 0, 0, 0
    n /= d
    
    alpha = compliance / (h*h)
    
    dlambda = - (C + alpha * prev_lambda) / (w1 + w2 + alpha)
    new_lambda = prev_lambda + dlambda
    dx = dlambda * n
    dx1 = w1 * dx
    dx2 = -w2 * dx
    
    return dx1, dx2, new_lambda

def static_collision_constraint(x1, x1_prev, normal, offset, h, compliance=0, prev_lambda=None):
    compliance = 0
    alpha = compliance / (h*h)
    
    # Check vertex-object intersection    
    d = -np.dot(normal, offset)
    ray = x1 - x1_prev
    ray_start = np.dot(x1_prev, normal) + d
    ray_projection = np.dot(ray, normal)
    
    if abs(ray_projection) < 1e-13:
        return 0, None, None
    
    t = - ray_start / ray_projection
    
    # No Collision 
    if t > 1:
        return 0, None, None

    # Continuous Collision
    if t > 0 and t <= 1:
        q_c = x1_prev + t * ray
    
        # C(x1) = (x1 - q) * n
        C = (x1 - q_c) * normal
        dC = normal
        dx = -C * dC
        return dx, None, None
    
    # Static collision
    if t <= 0:
        q_s = x1 - ray_projection * normal
        C = (x1 - q_s) * normal
        dC = normal
        dx = -C * dC
        return dx, None, None

# Simulation parameters
gravity = np.array([0, -9.81, 0])
time_step = 1/30
substeps = 1
h = time_step/substeps


cube1.currPos = cube1.geometry.attributes['position'].array.copy()
cube1.currVel = np.zeros(cube1.currPos.shape)
while True:
    ## Gather Collisions Candidates
    coll_idx = np.where(cube1.currPos[:, 1]<0)[0]
    for _ in range(substeps):
        ## Check Actual Collisions
        
        ## integration
        cube1.prevPos = cube1.currPos.copy()
        cube1.currVel += gravity * h * 1.0
        cube1.currPos += cube1.currVel * h
            
        # Solve Distance Constraints
        for i, j in cube1.edges:
            x1, x2 = cube1.currPos[i], cube1.currPos[j]
            rest_length = 1 if i != 8 and j != 8 else np.sqrt(3)/2
            dx1, dx2, _ = solve_distance_constraint(x1, x2, 1, 1, rest_length=rest_length, compliance=0.0, h=h, prev_lambda=0)
            cube1.currPos[i] += dx1
            cube1.currPos[j] += dx2         

        # Solve Volume Constraints
        for i, j, k, l in cube1.tet_indices:
            x1, x2, x3, x4 = cube1.currPos[i], cube1.currPos[j], cube1.currPos[k], cube1.currPos[l]
            rest_volume = 1/8
            print(rest_volume)
            dx1, dx2, dx3, dx4, _ = solve_volume_constraint(x1, x2, x3, x4, 1, 1, 1, 1, rest_volume=rest_volume, compliance=0.0, h=h, prev_lambda=0)
            cube1.currPos[i] += dx1
            cube1.currPos[j] += dx2
            cube1.currPos[k] += dx3
            cube1.currPos[l] += dx4
        
        ## Solve Collisons
        for i in coll_idx:
            x1 = cube1.currPos[i]
            x1_prev = cube1.prevPos[i]
            normal = np.array([0, 1, 0])
            offset = np.array([0, 0, 0])

            dx, _, _ = static_collision_constraint(x1, x1_prev, normal, offset, h, compliance=0, prev_lambda=0)

            cube1.currPos[i] += dx
        
        ## Updata Velocities with solved positions
        cube1.currVel = (cube1.currPos - cube1.prevPos) / h
            
        ## Solve Velocities  - Damping and Friction
        for i in coll_idx:          
            
            v_n = np.dot(cube1.currVel[i], normal) * normal
            v_t = cube1.currVel[i] - v_n
            cube1.currVel[i] = -v_n * 0.9 + v_t * 0.5
            
    cube1.geometry.attributes['position'].array = cube1.currPos.copy()
    
    
    time.sleep(0.033)

0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.12

KeyboardInterrupt: 

In [ ]:
def get_bounding_box(body):
    min = np.array([np.inf, np.inf, np.inf])
    max = np.array([-np.inf, -np.inf, -np.inf])
    for vertex in body.geometry.attributes['position'].array:
        min = np.minimum(min, vertex)
        max = np.maximum(max, vertex)
    return min, max

def check_collision(body1, body2):
    min1, max1 = get_bounding_box(body1)
    min2, max2 = get_bounding_box(body2)
    return min1[0] <= max2[0] and max1[0] >= min2[0] and min1[1] <= max2[1] and max1[1] >= min2[1] and min1[2] <= max2[2] and max1[2] >= min2[2]

def collect_collisions(bodies, plane):
    collisions = []
    # Check collision body vs body
    for i in range(len(bodies)):
        for j in range(i+1, len(bodies)):
            if check_collision(bodies[i], bodies[j]):
                collisions.append((bodies[i], bodies[j]))
    
    # Check collision body vs plane
    for body in bodies:
        if np.any(np.dot(body.geometry.attributes['position'].array, plane.normal) < 0):
            ground_collisions.append((body, plane))

    return collisions, ground_collisions

def check_contacts(collision, ground_collisions):
    contacts = []
    for body, plane in ground_collisions:
        n, o = plane.normal, plane.origin
            
        v = (body.currPos - body.prevPos)
        t = - (np.dot(n, body.prevPos) + o) / np.dot(n, v)
    
        q_c = body.prevPos + t * v
        q_s = body.prevPos - (np.dot(n, body.prevPos) + o) * n
        
        cont = np.where((0<t) and (t<=1))[0]
        stat = np.where(t<=0)[0]
        
        contacts.extend([(body, i, q_c[i], n, 'continuous') for i in cont])
        contacts.extend([(body, i, q_s[i], n, 'static') for i in stat])     
        
        return contacts
    
def solve_contacts(contacts):
    for (body, i, q, n, coll_type) in contacts:
        if coll_type == 'continuous':
            solve_continuous_contact(body, i, q, n)
        elif coll_type == 'static':
            solve_static_contact(body, i, q, n)
        
    
    def solve_positions(body):
        
        



In [ ]:
# Simulation parameters
gravity = np.array([0, -9.81, 0])
time_step = 1/30
substeps = 5
h = time_step/substeps
while True:
    ## Gather Collisions Candidates
    collisions, ground_collisions = collect_collisions(bodies, plane) # -> the pair of bodies
    
    for _ in range(substeps):
        ## Check Actual Collisions
        contacts = check_contacts(collisions, ground_collisions) # -> constraint generate : 
        
        for body in bodies:
            integrate(body, gravity, h) # update position and orientation with external forces and torques
            
        ## Solve Constraints : distance constraints (attach, hinge, etc.)
        # for constraint in constraints:
        #     constraint.solve(h)
            
        for body in bodies:
            for (x1, x2, rest_length, curr_lambda) in body.distance_constraints:
                solve_distance_constraint(x1, x2, 1, 1, rest_length=rest_length, compliance=0.0, h=h, prev_lambda=curr_lambda)
        ## Solve Collisons
        solve_contacts(contacts)
        
        ## Updata Velocities with solved positions
        for body in bodies:
            update_velocities()
            
        ## Solve Velocities  - Damping and Friction
        solve_velocities()
    
    update_mesh(bodies)
    time.sleep(0.01)

In [ ]:
def dynamic_collision_constraint(x1, x2, w1, w2, n, d, compliance, h, prev_lambda=None):
    pass


def static_collision_constraint(x1, x1_prev, normal, offset, h, compliance=0, prev_lambda=None):
    compliance = 0
    alpha = compliance / (h*h)
    
    # Check vertex-object intersection    
    d = -np.dot(normal, offset)
    ray = x1 - x1_prev
    ray_start = np.dot(x1_prev, normal) + d
    ray_projection = np.dot(ray, normal)
    
    
    t = - ray_start / ray_projection
    
    # No Collision 
    if t > 1:
        return 0, None, None

    # Continuous Collision
    if t > 0 and t <= 1:
        q_c = x1_prev + t * ray
    
        # C(x1) = (x1 - q) * n
        C = (x1 - q_c) * normal
        dC = normal
        dx = -C * dC
        return dx, None, None
    
    # Static collision
    if t <= 0:
        q_s = x1 - ray_projection * normal
        C = (x1 - q_s) * normal
        dC = normal
        dx = -C * dC
        return dx, None, None

def solve_distance_constraint(x1, x2, w1, w2, rest_length, compliance, h, prev_lambda=None):
    n = (x1 - x2)
    d = np.linalg.norm(n)
    # C(x1, x2) = |x1 - x2| - rest_length
    C = d - rest_length 
    
    if d < 1e-6:
        return 0, 0, 0
    n /= d
    
    alpha = compliance / (h*h)
    
    dlambda = - (C + alpha * prev_lambda) / (w1 + w2 + alpha)
    new_lambda = prev_lambda + dlambda
    dx = dlambda * n
    dx1 = w1 * dx
    dx2 = -w2 * dx
    
    return dx1, dx2, new_lambda


